In [19]:
from detector import ModifiedHoughCropRowDetector, SplitLawinVegetationDetector, HoughCropRowDetector, get_medians
from PIL import Image
from scipy.stats import kstest
import plotly.express as px
import torch.nn.functional as F
import os
import math
import torch
import numpy as np
import numpy as np
import cv2 as cv

In [37]:
base = "../Datasets/WeedMap/0_rotations_processed_003_test/RedEdge/000/tile"
img_name_1 = "frame0046_0.png"
img_name_2 = "frame0048_3.png"
channels = ["R", "G", "B", "NIR", "RE"]
img_channels1 = [np.array(Image.open(os.path.join(base, c, img_name_1))) for c in channels]
img_channels2 = [np.array(Image.open(os.path.join(base, c, img_name_2))) for c in channels]
img1 = np.stack(img_channels1)
img2 = np.stack(img_channels2)

In [38]:
def get_drawn_img(img, theta_rho):
    draw_img = np.array(img[:3].transpose(1, 2, 0)).copy()
    for i in range(0, len(theta_rho)):
        rho = theta_rho[i][0]
        theta = theta_rho[i][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(draw_img, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
    return Image.fromarray(draw_img)

In [39]:
segmentor = SplitLawinVegetationDetector()
crd = HoughCropRowDetector(crop_detector=segmentor, threshold=50)

Some weights of the model checkpoint at nvidia/segformer-b1-finetuned-ade-512-512 were not used when initializing SegformerModel: ['decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.weight', 'decode_head.batch_norm.running_var', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.classifier.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.3.proj.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.batch_norm.bias', 'decode_head.batch_norm.running_mean']
- This IS expected if you are initializing SegformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Segforme

In [83]:
tensor = torch.from_numpy(img1).float() / 255
crop_mask = crd.detect_crop(tensor)
lines = crd.hough(crop_mask)

In [84]:
px.histogram(x=lines[:, 1], nbins=100, cumulative=False).show()

In [85]:
px.histogram(x=lines[:, 1], nbins=100, cumulative=True).show()

In [86]:
kstest((lines[:, 1] - lines[:, 1].min()) / (lines[:, 1].max() - lines[:, 1].min()), "uniform")

KstestResult(statistic=0.4462978706714955, pvalue=3.9775770498625673e-26)

In [71]:
tensor = torch.from_numpy(img2).float() / 255
crop_mask = crd.detect_crop(tensor)
lines = crd.hough(crop_mask)

In [72]:
px.histogram(x=lines[:, 1], nbins=100).show()

In [73]:
px.histogram(x=lines[:, 1], nbins=100, cumulative=True).show()

In [87]:
kstest((lines[:, 1] - lines[:, 1].min()) / (lines[:, 1].max() - lines[:, 1].min()), "uniform").statistic

0.4462978706714955

In [81]:
(lines[:, 1] - lines[:, 1].min()) / (lines[:, 1].max() - lines[:, 1].min())

tensor([0.7542, 0.7542, 0.7542,  ..., 0.8939, 0.8994, 0.9218])